# DeepANM: End-to-End Demo (Sachs Dataset)

Notebook này minh họa cách sử dụng **DeepANM** để khám phá quan hệ nhân quả trên tập dữ liệu protein signaling (Sachs et al., 2005).

### Quy trình:
1. Load dữ liệu và tiền xử lý.
2. Huấn luyện DeepANM.
3. Trích xuất quan hệ nhân quả (DAG).
4. Visualizing kết quả (Heatmap & Clusters).

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import QuantileTransformer
from deepanm import DeepANM

# Setup
np.random.seed(42)
torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

## 1. Load và Tiền xử lý dữ liệu

In [ ]:
data = np.load('../data/sachs/continuous/data1.npy')
headers = np.load('../data/sachs/sachs-header.npy')

qt = QuantileTransformer(output_distribution='normal', n_quantiles=500)
data_norm = qt.fit_transform(data)

print(f"Data shape: {data_norm.shape}")
print(f"Proteins: {headers}")

## 2. Huấn luyện mô hình DeepANM
Chúng ta sẽ để mô hình tự học ma trận kề (DAG) thông qua NOTEARS penalty kết hợp với HSIC.

In [ ]:
model = DeepANM(data=data_norm, n_clusters=2, epochs=200, lda=1.0, device=device)
W_raw, W_bin = model.get_dag_matrix(threshold=0.15)

## 3. Visualization: Ma trận kề (Causal Graph)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(W_bin, annot=True, xticklabels=headers, yticklabels=headers, cmap="Blues")
plt.title("Learned Causal Structure (DeepANM)")
plt.xlabel("Effect (Child)")
plt.ylabel("Cause (Parent)")
plt.show()

## 4. Phân cụm Cơ chế (Mechanism Discovery)
DeepANM có khả năng phát hiện xem các điểm dữ liệu thuộc về những cơ chế nhân quả khác nhau như thế nào.

In [ ]:
clusters = model.predict_clusters(data_norm)

# Plotting 2 protein bất kỳ để xem phân cụm
idx1, idx2 = 0, 1 # Raf, Mek
plt.figure(figsize=(8, 6))
plt.scatter(data_norm[:, idx1], data_norm[:, idx2], c=clusters, cmap='viridis', alpha=0.5)
plt.xlabel(headers[idx1])
plt.ylabel(headers[idx2])
plt.title(f"Mechanism Clustering: {headers[idx1]} vs {headers[idx2]}")
plt.colorbar(label='Mechanism ID')
plt.show()